# Gradient Descent

Gradient descent is a simple algorithm at the heart of many (if not most) machine learning algorithms and concepts.
Just like in calculus, the goal is to use a function's derivative (gradient) to find an extreme point (usually a global minimum) for that function.

In this example, we will cover:
 - The basic concept of gradient descent.
 - An aside on plotting functions.
 - What gradient descent looks like on a simple function.
 - How different learning rates change gradient descent's behavior.

For this exercise, we will be working with the simple function:
$$
    f(x) = x^2 - 4x
$$

Keep in mind that most applications in machine learning will be dealing with many more dimensions,
but the same concepts will apply.

Let's begin by getting some data that follows our function.

In [ ]:
import numpy
import matplotlib.pyplot

# Get 1000 evenly spaced points between -2 and 6.
# (we chose those numbers so they will look good on a plot for this specific function).
X = numpy.linspace(-2, 6, 1000)

# f(x) = x^2 - 4x
# Note that this function works for both scalars and numpy.ndarrays.
def f(x):
    return (x ** 2) - (4 * x)

# Construct the y values from the x values.
Y = f(X)

print(X[0:20])
print(Y[0:20])

Let's plot our function.

In [ ]:
matplotlib.pyplot.plot(X, Y)
matplotlib.pyplot.show()

## An Aside on Plotting Functions

In general, computers are not good at handling abstract mathematical equations/functions.
Mathematical equations are for people, computers want data points.
So when we graph "functions", what we are typically doing is evaluating that function at many input values and creating a list of data points that can then be plotted.
With enough data points, the resulting graph can look smooth/accurate.
Even your old pal, the TI-83/89 calculators, did not graph lines directly.

See below how the number of points affects how smooth the plot looks.

In [ ]:
number_of_points = [2, 4, 8, 16, 32, 64]

for count in number_of_points:
    x_test = numpy.linspace(-2, 6, count)
    y_test = f(x_test)
    
    matplotlib.pyplot.plot(x_test, y_test)
    matplotlib.pyplot.title("%d Points" % (count))
    matplotlib.pyplot.show()

## Back to Gradient Descent

Now that we have our function plotted, let's see how gradient descent can get us to the minimum.

Let's choose a starting point $ x_0 = -1 $.

In [ ]:
x_0 = -1

matplotlib.pyplot.plot(X, Y)
matplotlib.pyplot.plot(x_0, f(x_0), 'rx')
matplotlib.pyplot.show()

Now we will need to compute the gradient of our function:
$$ \begin{align}
    f(x)        &= x^2 - 4x \\
    \nabla f(x) &= 2x - 4 \\
\end{align} $$

In [ ]:
# f'(x) = 2x - 4
def gradient_f(x):
    return (2 * x) - 4

print("Starting point (x_0): (%f, %f)" % (x_0, f(x_0)))
print("Gradient at x_0: %f" % (gradient_f(x_0)))

Recall that gradient descent computes the next point ($ x_{n + 1} $) using the equation:
$$
    x_{n + 1} = x_n - \alpha \nabla f(x_n)
$$
Where $ \alpha $ is the *learning rate*, which decides how far we will move in one step.

Let's arbitrarily pick a learning rate of 0.1.
(Note that this learning rate is not actually arbitrary, I know that it will work well. We will cover setting a learning rate later.)

With our starting point, learning rate, and gradient all figured out,
we can compute the next point to visit.

$$ \begin{align}
    x_{n + 1} &= x_n - \alpha \nabla f(x_n) \\
    x_{1}     &= -1 - (0.1 * \nabla f(x_0) \\
    x_{1}     &= -1 - (0.1 * (2 x_0 - 4) \\
    x_{1}     &= -1 - (0.1 * (2 * -1) - 4) \\
    x_{1}     &= -1 - (0.1 * -6) \\
    x_{1}     &= -1 - -0.6 \\
    x_{1}     &= -0.4 \\
\end{align} $$

Therefore, we know that the next $ x $ we will visit is $ -0.4 $,
where our function has a value of $ f(-0.4) = 1.76 $
(which is lower than the $ f(-1) = 3.0 $ that we were at in the initial location).

We can also do all the above in code:

In [ ]:
alpha = 0.1
x_1 = x_0 - (alpha * gradient_f(x_0))

print("Next point (x_1): (%f, %f)" % (x_1, f(x_1)))

matplotlib.pyplot.plot(X, Y)
matplotlib.pyplot.plot(x_0, f(x_0), 'rx')
matplotlib.pyplot.plot(x_1, f(x_1), 'rx')
matplotlib.pyplot.show()

We have moved closer to the minimum!

Now that we have done this process one by hand, let's make a function to do it for us:

In [ ]:
def gradient_descent(function, gradient, starting_point,
                     learning_rate = 0.1, num_steps = 20, quiet = True):
    # Keep track of all the points we visited.
    history = [starting_point]
    
    x = starting_point
    
    if (not quiet):
        print("Starting at (%f, %f)" % (x, function(x)))
          
    for step in range(num_steps):
        x_next = x - (learning_rate * gradient(x))
        
        if (not quiet):
            print("Moved to (%f, %f)" % (x_next, function(x_next)))
        
        history.append(x_next)
        x = x_next
    
    return history

This function will do `num_steps` iterations of gradient descent for us and return a list of all the points visited.
(Note that this implementation relies on function objects that are passed in for the base and gradient functions, which means it is not specific to $ f(x) = x^2 - 4x $.)

Let's try our our new gradient descent implementation.

In [ ]:
history = gradient_descent(f, gradient_f, x_0, learning_rate = alpha, quiet = False)

matplotlib.pyplot.plot(X, Y)
matplotlib.pyplot.plot(history, f(numpy.array(history)), 'rx')
matplotlib.pyplot.show()

Fantastic!
It looks like we were able to make it all the way to the minimum of $ f(x) $.

## Learning Rates

Choosing a good learning rate for gradient descent can be pretty important to how well it will function.
If you pick a learning rate that is too low, gradient descent can take a long time to find the right answer.
But if you pick a learning rate that is too large, gradient descent can behave strangely and maybe never find a good answer.

To see this in action, let's try our a few different learning rates.
But first, we will make a function to speed up our testing.

In [ ]:
def test_learning_rate(learning_rate, num_steps = 20, quiet = True):
    history = gradient_descent(f, gradient_f, x_0,
                               learning_rate = learning_rate, num_steps = num_steps,
                               quiet = quiet)

    matplotlib.pyplot.plot(X, Y)
    matplotlib.pyplot.plot(history, f(numpy.array(history)), 'rx')
    matplotlib.pyplot.title("Learning Rate: %f, Steps: %d" % (learning_rate, num_steps))
    matplotlib.pyplot.show()

Now we will start with a learning rate that we have already seen will work well.

In [ ]:
test_learning_rate(0.1)

Looks fine.
What happens when we decrease the learning rate.

In [ ]:
test_learning_rate(0.01)

We don't make it all the way down to the minimum.
We will have to do more steps if we want to get all the way to the minimum.

In [ ]:
test_learning_rate(0.01, num_steps = 40)

Because of the decreasing nature of the gradient, even through it looks like 20 steps got us half the way we need far more than double the number of steps to get all the way to the minimum.

In [ ]:
test_learning_rate(0.01, num_steps = 200)

Now let's see what happens with a larger learning rate.

In [ ]:
test_learning_rate(1.0, quiet = False)

That's pretty bad ...

With this learning rate, gradient descent just keeps bouncing around between (-1, 5) and (5, 5).
With this learning rate, we will never converge to an answer.

If we set the learning rate just slightly higher, it gets even worse.

In [ ]:
test_learning_rate(1.01, quiet = False)

We will just keep moving further away from our goal.

With a slightly smaller step size we will get closer to the minimum, but very slowly.

In [ ]:
test_learning_rate(0.99, num_steps = 100)

When we choose a step size that is too large but still reasonable,
then we may overshoot the minimum, but eventually converge to it.

In [ ]:
test_learning_rate(0.75, quiet = False)